In [1]:
from pathlib import Path
import re

from pysam import FastaFile
import pandas as pd

In [2]:
DATA_ROOT = Path('/Users/liang/Box/MyCPTAC/CPTAC_proteome_v2.1/')

refseq_fasta_pth = DATA_ROOT / 'DCC/RefSeq_20180629/RefSeq.20180629_Human_ucsc_hg38_cpdbnr_mito_264contams.fasta.gz'

phospho_pth = DATA_ROOT / 'CPTAC3/UCEC_discovery/PNNL_with_refseq/ucec_phospho_d6_refseq_B12345_with_sites.txt.gz'
acetyl_pth = DATA_ROOT / 'CPTAC3/UCEC_discovery/PNNL_with_refseq/ucec_acetyl_d6_refseq_B12345_with_sites.txt.gz'

In [3]:
protein_fa = FastaFile(str(refseq_fasta_pth))

In [4]:
def calc_peptide_start_end(peptide, protein_id, sites):
    # Make sure the peptide is part of the protein sequence
    protein_seq = protein_fa.fetch(protein_id)
    if not peptide.upper() in protein_seq:
        return [None, None, False, 'Peptide not found in protein']
    
    first_res = sites[0][0].lower()
    first_res_loc = int(sites[0][1:])
    first_res_ix = peptide.find(first_res) 
    
    peptide_start = first_res_loc - first_res_ix
    peptide_end = first_res_loc + len(peptide) - first_res_ix - 1
    if protein_seq[peptide_start - 1:peptide_end] != peptide.upper():
        return [None, None, False, 'Invalid location']
    return [peptide_start, peptide_end, True, None]

## Phospho

In [5]:
raw_phospho_df = pd.read_table(phospho_pth)
raw_phospho_df.columns

Index(['Gene', 'RefSeq', 'peptide', 'site', 'CPT000073.0011', 'CPT000082.0012',
       'CPT000098.0012', 'CPT000114.0003', 'CPT000130.0009', 'CPT000142.0009',
       ...
       'CPT019126.0002', 'CPT022985.0002', 'CPT022989.0002', 'CPT022995.0002',
       'CPT023003.0002', 'CPT023013.0002', 'CPT023019.0002', 'CPT023025.0002',
       'CPT023040.0002', 'CPT023046.0002'],
      dtype='object', length=157)

In [6]:
raw_phospho_df.head()

,Gene,RefSeq,peptide,site,CPT000073.0011,CPT000082.0012,CPT000098.0012,CPT000114.0003,CPT000130.0009,CPT000142.0009,...,CPT019126.0002,CPT022985.0002,CPT022989.0002,CPT022995.0002,CPT023003.0002,CPT023013.0002,CPT023019.0002,CPT023025.0002,CPT023040.0002,CPT023046.0002
0,AAAS,NP_056480.1,AQEPPAGGGGSIHDLPLFTETSPTSAPWDPLPGPPPVLPHS*PHSHL,AAAS-S541s,NaN,NaN,-0.0461,NaN,NaN,NaN,...,-0.1220,0.0681,-0.301,-0.191,-0.1870,-0.191,-0.517,-0.0108,0.256,-0.1580
1,AAAS,NP_056480.1,FS*PVLGR,AAAS-S495s,NaN,0.25,0.5560,0.273,NaN,-0.202,...,0.0421,-0.5100,0.215,-0.359,0.0606,-0.272,0.223,-0.3940,0.149,0.0774
2,AAAS,NP_056480.1,IAHIPLY*FVNAQFPRFSPVLGR,AAAS-Y485y,NaN,NaN,NaN,-0.407,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAAS,NP_056480.1,IAHIPLYFVNAQFPRFS*PVLGR,AAAS-S495s,NaN,NaN,NaN,-0.277,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AACS,NP_076417.2,HVPS*LILETK,AACS-S618s,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
peptide_df = raw_phospho_df.loc[:, ['Gene', 'RefSeq', 'peptide', 'site']]
peptide_df['original_id'] = peptide_df['peptide']
peptide_df['peptide'] = peptide_df['peptide'].str.replace(r'([STY])\*', lambda m: m.group(1).lower())
peptide_df['phosphosites'] = peptide_df['site'].str.split('-', 1, expand=True).iloc[:, 1].str.findall(r'[A-Z]\d+[a-z]')
peptide_df.head(10)

,Gene,RefSeq,peptide,site,original_id,phosphosites
0,AAAS,NP_056480.1,AQEPPAGGGGSIHDLPLFTETSPTSAPWDPLPGPPPVLPHsPHSHL,AAAS-S541s,AQEPPAGGGGSIHDLPLFTETSPTSAPWDPLPGPPPVLPHS*PHSHL,[S541s]
1,AAAS,NP_056480.1,FsPVLGR,AAAS-S495s,FS*PVLGR,[S495s]
2,AAAS,NP_056480.1,IAHIPLyFVNAQFPRFSPVLGR,AAAS-Y485y,IAHIPLY*FVNAQFPRFSPVLGR,[Y485y]
3,AAAS,NP_056480.1,IAHIPLYFVNAQFPRFsPVLGR,AAAS-S495s,IAHIPLYFVNAQFPRFS*PVLGR,[S495s]
4,AACS,NP_076417.2,HVPsLILETK,AACS-S618s,HVPS*LILETK,[S618s]
5,AAED1,NP_714542.1,QVsGAAALVPAPSGPDSGQPLAAAVAELPVLDAR,AAED1-S12s,QVS*GAAALVPAPSGPDSGQPLAAAVAELPVLDAR,[S12s]
6,AAGAB,NP_078942.3,AFWMAIGGDRDEIEGLsSDEEH,AAGAB-S310s,AFWMAIGGDRDEIEGLS*SDEEH,[S310s]
7,AAGAB,NP_078942.3,AFWMAIGGDRDEIEGLSsDEEH,AAGAB-S311s,AFWMAIGGDRDEIEGLSS*DEEH,[S311s]
8,AAK1,NP_055726.3,ADVAVEsLIPGLEPPVPQRLPSQTESVTSNR,AAK1-S831s,ADVAVES*LIPGLEPPVPQRLPSQTESVTSNR,[S831s]
9,AAK1,NP_055726.3,ADVAVESLIPGLEPPVPQRLPsQTESVTSNR,AAK1-S846s,ADVAVESLIPGLEPPVPQRLPS*QTESVTSNR,[S846s]


In [8]:
# Check what residues are modified
peptide_df['phosphosites'].explode().apply(lambda x: x[0]).value_counts()

S    118489
T     27319
Y      4048
Name: phosphosites, dtype: int64

In [9]:
# Keep only STY that are phosphosites
peptide_df['phosphosites'] = peptide_df['phosphosites'].apply(
    lambda l: [x[:-1] for x in l if x[0] in 'STY']
)

In [10]:
# Calculate the peptide location
peptide_loc_range_df = pd.DataFrame(
    map(calc_peptide_start_end, 
        peptide_df.peptide, peptide_df.RefSeq, peptide_df.phosphosites),
    index=peptide_df.index,
    columns=['peptide_start', 'peptide_end', 'peptide_loc_valid', 'peptide_loc_invalid_reason'],
).astype({
    'peptide_start': pd.Int64Dtype(),
    'peptide_end': pd.Int64Dtype(),
})

In [11]:
# Combine the location and peptide information
peptide_with_loc_df = pd.concat([peptide_df, peptide_loc_range_df], axis=1)
# List all the invalid peptides
peptide_with_loc_df.loc[
    ~peptide_loc_range_df['peptide_loc_valid'],
    ['RefSeq', 'phosphosites', 'peptide', 'peptide_loc_invalid_reason']
]

,RefSeq,phosphosites,peptide,peptide_loc_invalid_reason


In [12]:
out_df = peptide_with_loc_df.loc[
    :,
    ['original_id', 'peptide', 'phosphosites', 'peptide_start', 'peptide_end', 'peptide_loc_valid', 'peptide_loc_invalid_reason']
]
out_df['phosphosites'] = out_df['phosphosites'].str.join(';')
out_df.to_csv(
    phospho_pth.with_suffix('.peptide_location.tsv.gz'),
    sep='\t',
    index=False
)

In [13]:
out_df.head()

,original_id,peptide,phosphosites,peptide_start,peptide_end,peptide_loc_valid,peptide_loc_invalid_reason
0,AQEPPAGGGGSIHDLPLFTETSPTSAPWDPLPGPPPVLPHS*PHSHL,AQEPPAGGGGSIHDLPLFTETSPTSAPWDPLPGPPPVLPHsPHSHL,S541,501,546,True,None
1,FS*PVLGR,FsPVLGR,S495,494,500,True,None
2,IAHIPLY*FVNAQFPRFSPVLGR,IAHIPLyFVNAQFPRFSPVLGR,Y485,479,500,True,None
3,IAHIPLYFVNAQFPRFS*PVLGR,IAHIPLYFVNAQFPRFsPVLGR,S495,479,500,True,None
4,HVPS*LILETK,HVPsLILETK,S618,615,624,True,None


## Acetylation

In [14]:
raw_acetyl_df = pd.read_table(acetyl_pth)
raw_acetyl_df.columns

Index(['Gene', 'RefSeq', 'peptide', 'site', 'CPT000073.0011', 'CPT000082.0012',
       'CPT000098.0012', 'CPT000114.0003', 'CPT000130.0009', 'CPT000142.0009',
       ...
       'CPT019126.0002', 'CPT022985.0002', 'CPT022989.0002', 'CPT022995.0002',
       'CPT023003.0002', 'CPT023013.0002', 'CPT023019.0002', 'CPT023025.0002',
       'CPT023040.0002', 'CPT023046.0002'],
      dtype='object', length=157)

In [15]:
peptide_df = raw_acetyl_df.loc[:, ['Gene', 'RefSeq', 'peptide', 'site']]
peptide_df['original_id'] = peptide_df['peptide'].copy()
peptide_df['peptide'] = (
    peptide_df['peptide']
        .str.replace(r'([A-Z])\#', lambda m: m.group(1).lower())
        # Drop additional symbols 
        .str.replace(r'([A-Z])\$', lambda m: m.group(1))
        .str.replace(r'([A-Z])\*', lambda m: m.group(1))
        .str.replace(r'([A-Z])\@', lambda m: m.group(1))
)
peptide_df['acetylsites'] = peptide_df['site'].str.split('-', 1, expand=True).iloc[:, 1].str.findall(r'[A-Z]\d+[a-z]')
peptide_df.head(10)

,Gene,RefSeq,peptide,site,original_id,acetylsites
0,A2M,NP_000005.2,AFTNSkIR,A2M-K682k,AFTNSK#IR,[K682k]
1,A2M,NP_000005.2,EVLkSLNEEAVK,A2M-K1168k,EVLK#SLNEEAVK,[K1168k]
2,A2M,NP_000005.2,NEDSLVFVQTDkSIYK,A2M-K135k,NEDSLVFVQTDK#SIYK,[K135k]
3,A2M,NP_000005.2,SIYKPGQTVkFR,A2M-K145k,SIYKPGQTVK#FR,[K145k]
4,A2M,NP_000005.2,SLNEEAVkK,A2M-K1176k,SLNEEAVK#K,[K1176k]
5,A2M,NP_000005.2,HNVYINGITYTPVSSTNEk,A2M-K664k,HNVYIN$GITYTPVSSTNEK#,[K664k]
6,A2M,NP_000005.2,TGTHGLLVkQEDMK,A2M-K516k,TGTHGLLVK#QEDM*K,[K516k]
7,A2M,NP_000005.2,TGTHGLLVkQEDMK,A2M-K516k,TGTHGLLVK#QEDMK,[K516k]
8,AACS,NP_076417.2,WLSVLEEkAMK,AACS-K391k,WLSVLEEK#AMK,[K391k]
9,AAGAB,NP_078942.3,VHAEkVAK,AAGAB-K290k,VHAEK#VAK,[K290k]


In [16]:
# Check what residues are modified
peptide_df['acetylsites'].explode().apply(lambda x: x[0]).value_counts()

K    14807
Name: acetylsites, dtype: int64

In [17]:
# Keep only K that are uniquitsites
peptide_df['acetylsites'] = peptide_df['acetylsites'].apply(
    lambda l: [x[:-1] for x in l if x[0] in 'K']
)

In [18]:
# Calculate the peptide location
peptide_loc_range_df = pd.DataFrame(
    map(calc_peptide_start_end, 
        peptide_df.peptide, peptide_df.RefSeq, peptide_df.acetylsites),
    index=peptide_df.index,
    columns=['peptide_start', 'peptide_end', 'peptide_loc_valid', 'peptide_loc_invalid_reason'],
).astype({
    'peptide_start': pd.Int64Dtype(),
    'peptide_end': pd.Int64Dtype(),
})

In [19]:
# Combine the location and peptide information
peptide_with_loc_df = pd.concat([peptide_df, peptide_loc_range_df], axis=1)
# List all the invalid peptides
peptide_with_loc_df.loc[
    ~peptide_loc_range_df['peptide_loc_valid'],
    ['RefSeq', 'acetylsites', 'peptide', 'original_id', 'peptide_loc_invalid_reason']
]

,RefSeq,acetylsites,peptide,original_id,peptide_loc_invalid_reason


In [20]:
out_df = peptide_with_loc_df.loc[
    :,
    ['original_id', 'peptide', 'acetylsites', 'peptide_start', 'peptide_end', 'peptide_loc_valid', 'peptide_loc_invalid_reason']
]
out_df['acetylsites'] = out_df['acetylsites'].str.join(';')
out_df.to_csv(
    acetyl_pth.with_suffix('.peptide_location.tsv.gz'),
    sep='\t',
    index=False
)

In [21]:
out_df.head()

,original_id,peptide,acetylsites,peptide_start,peptide_end,peptide_loc_valid,peptide_loc_invalid_reason
0,AFTNSK#IR,AFTNSkIR,K682,677,684,True,None
1,EVLK#SLNEEAVK,EVLkSLNEEAVK,K1168,1165,1176,True,None
2,NEDSLVFVQTDK#SIYK,NEDSLVFVQTDkSIYK,K135,124,139,True,None
3,SIYKPGQTVK#FR,SIYKPGQTVkFR,K145,136,147,True,None
4,SLNEEAVK#K,SLNEEAVkK,K1176,1169,1177,True,None
